# Enhanced Geospatial EDA for NYC Taxi Data
## Using Dask, GeoPandas, H3, and Advanced Visualization

This notebook performs comprehensive exploratory data analysis on NYC taxi data with focus on geospatial patterns, temporal trends, and advanced visualizations.

In [1]:
# Import all necessary libraries
import warnings
warnings.filterwarnings('ignore')

# Core data processing
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster
import pandas as pd
import numpy as np

# Geospatial libraries
import geopandas as gpd
import h3
from shapely.geometry import Point, Polygon
from shapely import wkt
import contextily as ctx

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import HeatMap, MarkerCluster, FastMarkerCluster
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

# Statistical analysis
from scipy import stats
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

# Utilities
import logging
from datetime import datetime, timedelta
import os
import json

# Configuration
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pio.templates.default = "plotly_white"

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

print("✅ All libraries imported successfully")

✅ All libraries imported successfully


In [2]:
# Configuration parameters
class Config:
    # Data paths
    RAW_TAXI_DATA_PATTERN = "../data/yellow_tripdata_*.csv" # Adjust if your data is elsewhere
    TAXI_ZONES_SHAPEFILE = "../data/taxi_zones/taxi_zones.shp" # Adjust if your data is elsewhere
    NYC_BOROUGHS_SHAPEFILE = "../data/boroughs/boroughs.shp" # Adjust if your data is elsewhere
    
    # Create dummy data and directories if they don't exist for notebook execution
    if not os.path.exists("../data"): os.makedirs("../data")
    if not os.path.exists("../data/taxi_zones"): os.makedirs("../data/taxi_zones")
    if not os.path.exists("../data/boroughs"): os.makedirs("../data/boroughs")
    
    # Create a dummy CSV file if none match the pattern
    import glob
    if not glob.glob(RAW_TAXI_DATA_PATTERN):
        print("Creating dummy taxi data for demonstration...")
        dummy_df = pd.DataFrame({
            'tpep_pickup_datetime': pd.to_datetime(['2023-01-01 10:00:00', '2023-01-01 10:15:00']),
            'tpep_dropoff_datetime': pd.to_datetime(['2023-01-01 10:10:00', '2023-01-01 10:30:00']),
            'pickup_longitude': [-73.985130, -73.990000],
            'pickup_latitude': [40.758896, 40.750000],
            'dropoff_longitude': [-73.995000, -73.980000],
            'dropoff_latitude': [40.740000, 40.760000],
            'passenger_count': [1.0, 2.0],
            'trip_distance': [1.5, 2.0],
            'fare_amount': [10.0, 12.5],
            'tip_amount': [2.0, 2.5],
            'total_amount': [15.0, 18.0],
            'RatecodeID': [1.0, 1.0],
            'VendorID': [1.0, 2.0],
            'payment_type': [1.0, 1.0] 
        })
        dummy_df.to_csv("../data/yellow_tripdata_dummy.csv", index=False)
    
    # Column names
    PICKUP_DATETIME_COL = 'tpep_pickup_datetime'
    DROPOFF_DATETIME_COL = 'tpep_dropoff_datetime'
    PICKUP_LAT_COL = 'pickup_latitude'
    PICKUP_LON_COL = 'pickup_longitude'
    DROPOFF_LAT_COL = 'dropoff_latitude'
    DROPOFF_LON_COL = 'dropoff_longitude'
    
    # H3 configuration
    H3_RESOLUTIONS = [7, 8, 9]  # Multiple resolutions for different analyses
    H3_MAIN_RESOLUTION = 8
    
    # NYC boundaries (approximate)
    NYC_BOUNDS = {
        'lat_min': 40.4774, 'lat_max': 40.9176,
        'lon_min': -74.2591, 'lon_max': -73.7004
    }
    
    # Dask configuration
    DASK_WORKERS = 2 # Reduced for local testing, adjust as needed
    DASK_THREADS_PER_WORKER = 2
    DASK_MEMORY_LIMIT = '2GB' # Reduced for local testing
    
    # Sampling rates for different analyses
    SAMPLE_RATES = {
        'visualization': 0.01,  # 1% for general viz
        'clustering': 0.005, # 0.5% for DBSCAN
        'heatmap': 0.1 # 10% for Folium Heatmap (can be higher for small datasets)
    }

config = Config()
print("✅ Configuration loaded")

✅ Configuration loaded


In [3]:
# Setup Dask Client with optimized configuration
def setup_dask_client():
    """Initialize Dask client with proper cleanup."""
    global client, cluster # Make them global to be accessible for cleanup
    try:
        # Clean up existing clients
        if 'client' in globals() and client:
            client.close()
        if 'cluster' in globals() and cluster:
            cluster.close()
    except (NameError, Exception) as e:
        logger.debug(f"No existing client to close: {e}")
    
    try:
        cluster = LocalCluster(
            n_workers=config.DASK_WORKERS,
            threads_per_worker=config.DASK_THREADS_PER_WORKER,
            memory_limit=config.DASK_MEMORY_LIMIT,
            dashboard_address=':8787', # Use a fixed port if possible, or None for auto
            silence_logs=logging.ERROR # Silence Dask's own INFO logs unless error
        )
        client = Client(cluster)
        logger.info(f"🚀 Dask Client initialized: {client.dashboard_link}")
        return client, cluster
    except Exception as e:
        logger.error(f"❌ Failed to initialize Dask client: {e}")
        # Fallback to a default client if LocalCluster setup fails
        try:
            client = Client()
            cluster = None # No explicit cluster object in this case
            logger.info(f"🚀 Dask Client initialized with defaults: {client.dashboard_link}")
            return client, cluster
        except Exception as e_fallback:
            logger.error(f"❌ Failed to initialize Dask with defaults: {e_fallback}")
            return None, None

client, cluster = setup_dask_client()
if client:
    print(f"Dask Dashboard: {client.dashboard_link}")
else:
    print("Dask client not available.")

2025-06-04 19:50:31,278 - INFO - 🚀 Dask Client initialized: http://127.0.0.1:53763/status


Dask Dashboard: http://127.0.0.1:53763/status


In [4]:
# Utility functions for data processing
class DataProcessor:
    @staticmethod
    def get_taxi_dtypes():
        """Define data types for taxi data to ensure proper loading."""
        return {
            'VendorID': 'float64',
            'passenger_count': 'float64',
            'trip_distance': 'float64',
            'RatecodeID': 'float64',
            'store_and_fwd_flag': 'object',
            config.PICKUP_LON_COL: 'float64',
            config.PICKUP_LAT_COL: 'float64',
            config.DROPOFF_LON_COL: 'float64',
            config.DROPOFF_LAT_COL: 'float64',
            'payment_type': 'float64',
            'fare_amount': 'float64',
            'extra': 'float64',
            'mta_tax': 'float64',
            'tip_amount': 'float64',
            'tolls_amount': 'float64',
            'improvement_surcharge': 'float64',
            'total_amount': 'float64',
            'congestion_surcharge': 'float64',
            'airport_fee': 'float64' # Might not be in older data
        }
    
    @staticmethod
    def filter_nyc_bounds(df, lat_col, lon_col):
        """Filter data to NYC boundaries."""
        return df[
            (df[lat_col].between(config.NYC_BOUNDS['lat_min'], config.NYC_BOUNDS['lat_max'])) &
            (df[lon_col].between(config.NYC_BOUNDS['lon_min'], config.NYC_BOUNDS['lon_max']))
        ]
    
    @staticmethod
    def safe_h3_convert(lat, lon, resolution):
        """Safely convert lat/lon to H3 hex."""
        try:
            if pd.isna(lat) or pd.isna(lon):
                return None
            # Validate coordinates before passing to h3
            if not (-90 <= float(lat) <= 90 and -180 <= float(lon) <= 180):
                return None
            return h3.geo_to_h3(float(lat), float(lon), int(resolution))
        except (ValueError, TypeError, Exception): # Catch h3.H3ValueError and other conversion issues
            return None
    
    @staticmethod
    def apply_h3_to_partition(df_partition, lat_col, lon_col, resolution, h3_col_name):
        """Apply H3 conversion to a Dask partition."""
        if lat_col not in df_partition.columns or lon_col not in df_partition.columns:
            df_partition[h3_col_name] = pd.Series([None] * len(df_partition), index=df_partition.index, dtype='object')
            return df_partition
        
        # Ensure lat/lon are numeric before applying
        df_partition[lat_col] = pd.to_numeric(df_partition[lat_col], errors='coerce')
        df_partition[lon_col] = pd.to_numeric(df_partition[lon_col], errors='coerce')

        df_partition[h3_col_name] = df_partition.apply(
            lambda row: DataProcessor.safe_h3_convert(row[lat_col], row[lon_col], resolution),
            axis=1
        )
        return df_partition

processor = DataProcessor()
print("✅ Data processing utilities loaded")

✅ Data processing utilities loaded


In [5]:
# Load and preprocess taxi data
def load_taxi_data():
    """Load taxi data with proper preprocessing."""
    logger.info("📊 Loading taxi data...")
    
    try:
        # Load data with specified dtypes
        dtypes = processor.get_taxi_dtypes()
        
        # Use all columns from dtypes, but only if they exist in CSV, handle missing ones
        # Dask's read_csv will try to infer types if not specified, or error if dtype mismatch
        # We provide 'assume_missing=True' to handle columns that might not be in all files
        ddf = dd.read_csv(
            config.RAW_TAXI_DATA_PATTERN,
            blocksize='64MB', # Smaller blocksize for potentially smaller files/memory
            dtype=dtypes,
            assume_missing=True, # Handles columns missing in some files
            # Following parameters help with parsing robustness
            # na_filter=False, # If you want to treat empty strings as strings not NaN, not typical for numeric data
            # skipinitialspace=True, # good for some CSVs
            # on_bad_lines='warn' # or 'skip' or a callable
        )
        
        # Ensure essential columns exist, fill with NaN if not (due to assume_missing)
        # This is more for Dask where a column might be entirely missing from a partition's meta
        # but assume_missing in read_csv should largely handle this.
        for col, dtype_str in dtypes.items():
            if col not in ddf.columns:
                if 'datetime' in col.lower(): 
                    ddf[col] = pd.NaT # For datetime
                elif dtype_str.startswith('float') or dtype_str.startswith('int'):
                    ddf[col] = np.nan # For numeric
                else:
                    ddf[col] = pd.NA # For object/string using pandas NA
        
        # Convert datetime columns
        date_cols = [config.PICKUP_DATETIME_COL, config.DROPOFF_DATETIME_COL]
        for col in date_cols:
            if col in ddf.columns:
                ddf[col] = dd.to_datetime(ddf[col], errors='coerce')
        
        # Ensure lat/lon columns are numeric
        coord_cols = [config.PICKUP_LAT_COL, config.PICKUP_LON_COL, 
                     config.DROPOFF_LAT_COL, config.DROPOFF_LON_COL]
        
        for col in coord_cols:
            if col in ddf.columns:
                # Check if conversion is needed to avoid re-converting if already numeric
                if not pd.api.types.is_numeric_dtype(ddf[col].dtype):
                    ddf[col] = dd.to_numeric(ddf[col], errors='coerce')
        
        logger.info(f"✅ Loaded {ddf.npartitions} partitions")
        logger.info(f"Columns: {list(ddf.columns)}")
        
        # Persist if client is available to speed up subsequent operations
        # if client: 
        #    logger.info("Persisting raw_taxi_ddf to memory...")
        #    ddf = ddf.persist()
            
        return ddf
        
    except Exception as e:
        logger.error(f"❌ Failed to load taxi data: {e}")
        # Create a minimal Dask DataFrame for the rest of the notebook to run if loading fails
        # This is useful for testing the notebook structure without full data
        logger.warning("Creating a minimal Dask DataFrame to proceed.")
        dummy_pdf = pd.DataFrame({
            config.PICKUP_DATETIME_COL: pd.to_datetime(['2023-01-01 10:00:00']),
            config.DROPOFF_DATETIME_COL: pd.to_datetime(['2023-01-01 10:10:00']),
            config.PICKUP_LAT_COL: [40.758896],
            config.PICKUP_LON_COL: [-73.985130],
            config.DROPOFF_LAT_COL: [40.740000],
            config.DROPOFF_LON_COL: [-73.995000],
            'passenger_count': [1.0],
            'trip_distance': [1.5]
        })
        # Ensure all expected columns from dtypes are present
        expected_cols = processor.get_taxi_dtypes()
        for col_name in expected_cols.keys():
            if col_name not in dummy_pdf.columns:
                if 'datetime' in col_name: dummy_pdf[col_name] = pd.NaT
                elif expected_cols[col_name].startswith('float'): dummy_pdf[col_name] = np.nan
                else: dummy_pdf[col_name] = None 
        return dd.from_pandas(dummy_pdf, npartitions=1)

# Load the data
raw_taxi_ddf = load_taxi_data()
if raw_taxi_ddf is not None:
    computed_length = raw_taxi_ddf.map_partitions(len).sum().compute()
    print(f"Data shape: {computed_length} rows")
    print(f"Sample data:")
    display(raw_taxi_ddf.head())
else:
    print("Taxi data loading failed, proceeding with minimal/no data.")

2025-06-04 19:50:42,776 - INFO - 📊 Loading taxi data...
2025-06-04 19:50:42,945 - INFO - ✅ Loaded 113 partitions
2025-06-04 19:50:42,946 - INFO - Columns: ['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'pickup_longitude', 'pickup_latitude', 'RateCodeID', 'store_and_fwd_flag', 'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'RatecodeID', 'congestion_surcharge', 'airport_fee']


Data shape: 47248845 rows
Sample data:


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,RatecodeID,congestion_surcharge,airport_fee
0,2.0,2015-01-15 19:05:39,2015-01-15 19:23:42,1.0,1.59,-73.993896,40.750111,1.0,N,-73.974785,40.750618,1.0,12.0,1.0,0.5,3.25,0.0,0.3,17.05,NaN,NaN,NaN
1,1.0,2015-01-10 20:33:38,2015-01-10 20:53:28,1.0,3.30,-74.001648,40.724243,1.0,N,-73.994415,40.759109,1.0,14.5,0.5,0.5,2.00,0.0,0.3,17.80,NaN,NaN,NaN
2,1.0,2015-01-10 20:33:38,2015-01-10 20:43:41,1.0,1.80,-73.963341,40.802788,1.0,N,-73.951820,40.824413,2.0,9.5,0.5,0.5,0.00,0.0,0.3,10.80,NaN,NaN,NaN
3,1.0,2015-01-10 20:33:39,2015-01-10 20:35:31,1.0,0.50,-74.009087,40.713818,1.0,N,-74.004326,40.719986,2.0,3.5,0.5,0.5,0.00,0.0,0.3,4.80,NaN,NaN,NaN
4,1.0,2015-01-10 20:33:39,2015-01-10 20:52:58,1.0,3.00,-73.971176,40.762428,1.0,N,-74.004181,40.742653,2.0,15.0,0.5,0.5,0.00,0.0,0.3,16.30,NaN,NaN,NaN


In [6]:
# Add H3 hexagon zones for different resolutions
def add_h3_zones(ddf):
    """Add H3 hexagon zones at multiple resolutions."""
    logger.info("🔷 Adding H3 zones...")
    
    processed_ddf = ddf.copy()
    
    # Check if coordinate columns exist
    if not all(col in ddf.columns for col in [config.PICKUP_LAT_COL, config.PICKUP_LON_COL]):
        logger.warning(f"❌ Coordinate columns ({config.PICKUP_LAT_COL}, {config.PICKUP_LON_COL}) not found. Skipping H3 zone addition.")
        return processed_ddf
    
    # Add H3 zones for each resolution
    for resolution in config.H3_RESOLUTIONS:
        h3_col = f'pickup_h3_r{resolution}'
        logger.info(f"Adding H3 resolution {resolution} as column '{h3_col}'...")
        
        # Define metadata for the new column explicitly
        meta_with_h3 = processed_ddf._meta.copy()
        meta_with_h3[h3_col] = 'object' 
        
        # Apply H3 conversion
        processed_ddf = processed_ddf.map_partitions(
            processor.apply_h3_to_partition,
            lat_col=config.PICKUP_LAT_COL,
            lon_col=config.PICKUP_LON_COL,
            resolution=resolution,
            h3_col_name=h3_col,
            meta=meta_with_h3 # Pass the new meta
        )
    
    # Add temporal features
    if config.PICKUP_DATETIME_COL in processed_ddf.columns and pd.api.types.is_datetime64_any_dtype(processed_ddf[config.PICKUP_DATETIME_COL].dtype):
        logger.info("Adding temporal features (hour, day_of_week, month, date)...")
        processed_ddf['pickup_hour'] = processed_ddf[config.PICKUP_DATETIME_COL].dt.hour
        processed_ddf['pickup_day_of_week'] = processed_ddf[config.PICKUP_DATETIME_COL].dt.dayofweek
        processed_ddf['pickup_month'] = processed_ddf[config.PICKUP_DATETIME_COL].dt.month
        processed_ddf['pickup_date'] = processed_ddf[config.PICKUP_DATETIME_COL].dt.date
    else:
        logger.warning(f"'{config.PICKUP_DATETIME_COL}' not found or not datetime. Skipping temporal feature addition.")
        # Add dummy columns if they don't exist to prevent errors later if code expects them
        for col in ['pickup_hour', 'pickup_day_of_week', 'pickup_month', 'pickup_date']:
            if col not in processed_ddf.columns:
                 processed_ddf[col] = np.nan 

    logger.info("✅ H3 zones and temporal features processed.")
    return processed_ddf

# Process the data
if raw_taxi_ddf is not None:
    processed_ddf = add_h3_zones(raw_taxi_ddf)
    
    # Persist processed_ddf if client is available
    # if client:
    #    logger.info("Persisting processed_ddf to memory...")
    #    processed_ddf = processed_ddf.persist()
    
    # Show sample with H3 zones
    h3_cols_to_show = [f'pickup_h3_r{res}' for res in config.H3_RESOLUTIONS if f'pickup_h3_r{res}' in processed_ddf.columns]
    cols_to_display = [config.PICKUP_LAT_COL, config.PICKUP_LON_COL] + h3_cols_to_show + ['pickup_hour', 'pickup_day_of_week']
    # Ensure columns exist before trying to display them
    cols_to_display = [col for col in cols_to_display if col in processed_ddf.columns]
    
    if cols_to_display:
        h3_sample = processed_ddf[cols_to_display].head()
        print("Sample with H3 zones and temporal features:")
        display(h3_sample)
    else:
        print("Relevant columns for H3 sample not found in processed_ddf.")
else:
    print("Skipping H3 zone addition as raw_taxi_ddf is not loaded.")
    processed_ddf = None # Ensure it's None if not processed

2025-06-04 19:52:38,117 - INFO - 🔷 Adding H3 zones...
2025-06-04 19:52:38,118 - INFO - Adding H3 resolution 7 as column 'pickup_h3_r7'...
2025-06-04 19:52:38,122 - INFO - Adding H3 resolution 8 as column 'pickup_h3_r8'...
2025-06-04 19:52:38,124 - INFO - Adding H3 resolution 9 as column 'pickup_h3_r9'...
2025-06-04 19:52:38,127 - INFO - Adding temporal features (hour, day_of_week, month, date)...
2025-06-04 19:52:38,140 - INFO - ✅ H3 zones and temporal features processed.


Sample with H3 zones and temporal features:


,pickup_latitude,pickup_longitude,pickup_h3_r7,pickup_h3_r8,pickup_h3_r9,pickup_hour,pickup_day_of_week
0,40.750111,-73.993896,None,None,None,19,3
1,40.724243,-74.001648,None,None,None,20,5
2,40.802788,-73.963341,None,None,None,20,5
3,40.713818,-74.009087,None,None,None,20,5
4,40.762428,-73.971176,None,None,None,20,5


In [7]:
# Load geospatial reference data
def load_geospatial_data():
    """Load NYC geospatial reference data."""
    geodata = {}
    
    # Try to load taxi zones
    try:
        if os.path.exists(config.TAXI_ZONES_SHAPEFILE):
            geodata['taxi_zones'] = gpd.read_file(config.TAXI_ZONES_SHAPEFILE)
            logger.info(f"✅ Loaded {len(geodata['taxi_zones'])} taxi zones")
        else:
            logger.warning(f"⚠️ Taxi zones shapefile not found at {config.TAXI_ZONES_SHAPEFILE}")
    except Exception as e:
        logger.error(f"❌ Failed to load taxi zones: {e}")
    
    # Try to load boroughs
    try:
        if os.path.exists(config.NYC_BOROUGHS_SHAPEFILE):
            geodata['boroughs'] = gpd.read_file(config.NYC_BOROUGHS_SHAPEFILE)
            logger.info(f"✅ Loaded {len(geodata['boroughs'])} boroughs")
        else:
            logger.warning(f"⚠️ Boroughs shapefile not found at {config.NYC_BOROUGHS_SHAPEFILE}")
    except Exception as e:
        logger.error(f"❌ Failed to load boroughs: {e}")
    
    # Create NYC boundary if no shapefiles available or if specified
    # Always create this as a fallback or general boundary
    logger.info("📍 Creating NYC boundary polygon from config.NYC_BOUNDS")
    bounds = config.NYC_BOUNDS
    nyc_polygon = Polygon([
        (bounds['lon_min'], bounds['lat_min']),
        (bounds['lon_max'], bounds['lat_min']),
        (bounds['lon_max'], bounds['lat_max']),
        (bounds['lon_min'], bounds['lat_max'])
    ])
    geodata['nyc_boundary'] = gpd.GeoDataFrame(
        {'name': ['NYC Config Boundary']}, 
        geometry=[nyc_polygon], 
        crs='EPSG:4326'
    )
    
    return geodata

# Load geospatial data
geo_data = load_geospatial_data()

# Display available geospatial data
for key, gdf in geo_data.items():
    print(f"--- {key} ---")
    if gdf is not None and not gdf.empty:
        print(f"  Features: {len(gdf)}")
        print(f"  Columns: {list(gdf.columns)}")
        print(f"  CRS: {gdf.crs}")
        display(gdf.head(2))
    else:
        print("  Not loaded or empty.")
    print()

2025-06-04 19:53:04,073 - INFO - ✅ Loaded 263 taxi zones
2025-06-04 19:53:04,081 - WARNING - ⚠️ Boroughs shapefile not found at ../data/boroughs/boroughs.shp
2025-06-04 19:53:04,082 - INFO - 📍 Creating NYC boundary polygon from config.NYC_BOUNDS


--- taxi_zones ---
  Features: 263
  Columns: ['OBJECTID', 'Shape_Leng', 'Shape_Area', 'zone', 'LocationID', 'borough', 'geometry']
  CRS: EPSG:2263


,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((933100.918 192536.086, 933091.011 19..."
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"MULTIPOLYGON (((1033269.244 172126.008, 103343..."



--- nyc_boundary ---
  Features: 1
  Columns: ['name', 'geometry']
  CRS: EPSG:4326


,name,geometry
0,NYC Config Boundary,"POLYGON ((-74.2591 40.4774, -73.7004 40.4774, ..."


In [13]:
len(processed_ddf)

ValueError: The columns in the computed data do not match the columns in the provided metadata.
  Extra:   []
  Missing: ['RateCodeID']

In [11]:
# Data Quality Assessment
def assess_data_quality(ddf):
    """Comprehensive data quality assessment."""
    if ddf is None:
        logger.warning("Input Dask DataFrame is None. Skipping data quality assessment.")
        return {
            'total_rows': 0,
            'quality_df': pd.DataFrame(columns=['Column', 'Null_Count', 'Null_Percentage']),
            'coord_stats': {}
        }
        
    logger.info("🔍 Assessing data quality...")
    
    # Basic statistics
    total_rows = len(ddf) # This will trigger a compute if not persisted
    if total_rows == 0:
        logger.warning("DataFrame is empty. Skipping detailed quality assessment.")
        return {
            'total_rows': 0,
            'quality_df': pd.DataFrame(columns=['Column', 'Null_Count', 'Null_Percentage']),
            'coord_stats': {}
        }
    logger.info(f"Total rows (computed): {total_rows:,}")
    
    # Missing values analysis
    null_counts = ddf.isnull().sum().compute()
    null_percentages = (null_counts / total_rows * 100).round(2)
    
    quality_df = pd.DataFrame({
        'Column': null_counts.index,
        'Null_Count': null_counts.values,
        'Null_Percentage': null_percentages.values
    })
    quality_df = quality_df[quality_df['Null_Count'] > 0].sort_values('Null_Percentage', ascending=False)
    
    # Coordinate validity check (using pickup locations)
    coord_stats = {}
    coord_cols_to_check = [(config.PICKUP_LAT_COL, 'lat_min', 'lat_max'), 
                           (config.PICKUP_LON_COL, 'lon_min', 'lon_max')]
    
    for col, min_bound_key, max_bound_key in coord_cols_to_check:
        if col in ddf.columns and pd.api.types.is_numeric_dtype(ddf[col].dtype):
            # Ensure bounds are valid before using them
            min_val = config.NYC_BOUNDS[min_bound_key]
            max_val = config.NYC_BOUNDS[max_bound_key]
            
            valid_coords = ddf[col].between(min_val, max_val).sum().compute()
            coord_stats[col] = {
                'valid_count': valid_coords,
                'valid_percentage': (valid_coords / total_rows * 100).round(2)
            }
        elif col in ddf.columns:
             logger.warning(f"Coordinate column {col} is not numeric, skipping validity check.")
        else:
            logger.warning(f"Coordinate column {col} not found, skipping validity check.")
            
    return {
        'total_rows': total_rows,
        'quality_df': quality_df,
        'coord_stats': coord_stats
    }

# Assess data quality
quality_assessment = assess_data_quality(processed_ddf)

print(f"📊 Data Quality Assessment")
print(f"Total rows: {quality_assessment['total_rows']:,}")
print("\n🔍 Missing Values:")
if not quality_assessment['quality_df'].empty:
    display(quality_assessment['quality_df'])
else:
    print("No missing values found or data is empty.")

print("\n📍 Coordinate Validity (Pickup points within NYC_BOUNDS):")
if quality_assessment['coord_stats']:
    for col, stats in quality_assessment['coord_stats'].items():
        print(f"  {col}: {stats['valid_count']:,} valid ({stats['valid_percentage']:.1f}%)")
else:
    print("Coordinate validity check not performed or no coordinate columns found.")

2025-06-04 20:28:15,813 - INFO - 🔍 Assessing data quality...


ValueError: The columns in the computed data do not match the columns in the provided metadata.
  Extra:   []
  Missing: ['RateCodeID']

In [10]:
# Temporal Analysis
def analyze_temporal_patterns(ddf):
    """Analyze temporal patterns in taxi trips."""
    if ddf is None or len(ddf) == 0:
        logger.warning("Input Dask DataFrame is None or empty. Skipping temporal analysis.")
        return {
            'hourly': pd.Series(dtype='int64'),
            'daily': pd.Series(dtype='int64'),
            'monthly': pd.Series(dtype='int64'),
            'hour_dow': pd.DataFrame()
        }
        
    logger.info("⏰ Analyzing temporal patterns...")
    
    # Ensure temporal columns exist and are of correct type
    required_temporal_cols = ['pickup_hour', 'pickup_day_of_week', 'pickup_month']
    if not all(col in ddf.columns for col in required_temporal_cols):
        logger.warning(f"Missing one or more temporal columns: {required_temporal_cols}. Aborting temporal analysis.")
        return {
            'hourly': pd.Series(dtype='int64'),
            'daily': pd.Series(dtype='int64'),
            'monthly': pd.Series(dtype='int64'),
            'hour_dow': pd.DataFrame()
        }

    # Filter valid datetime data (implicitly done by feature engineering, but good to be aware)
    # valid_datetime_ddf = ddf.dropna(subset=[config.PICKUP_DATETIME_COL]) # if using original datetime col
    valid_temporal_ddf = ddf.dropna(subset=required_temporal_cols) # Using derived features
    
    if len(valid_temporal_ddf) == 0:
        logger.warning("No valid temporal data after dropping NaNs. Skipping temporal analysis.")
        return {
            'hourly': pd.Series(dtype='int64'),
            'daily': pd.Series(dtype='int64'),
            'monthly': pd.Series(dtype='int64'),
            'hour_dow': pd.DataFrame()
        }

    # Hourly patterns
    hourly_trips = valid_temporal_ddf.groupby('pickup_hour').size().compute().sort_index()
    
    # Daily patterns
    daily_trips = valid_temporal_ddf.groupby('pickup_day_of_week').size().compute().sort_index()
    
    # Monthly patterns
    monthly_trips = valid_temporal_ddf.groupby('pickup_month').size().compute().sort_index()
    
    # Hour vs Day of Week
    hour_dow_data = valid_temporal_ddf.groupby(['pickup_hour', 'pickup_day_of_week']).size().compute()
    if not hour_dow_data.empty:
        hour_dow_data = hour_dow_data.unstack(fill_value=0).sort_index()
    else:
        hour_dow_data = pd.DataFrame() # Empty dataframe if no data
        
    return {
        'hourly': hourly_trips,
        'daily': daily_trips,
        'monthly': monthly_trips,
        'hour_dow': hour_dow_data
    }

# Analyze temporal patterns
temporal_patterns = analyze_temporal_patterns(processed_ddf)

# Create temporal visualizations
if not all(val.empty for val in temporal_patterns.values()):
    fig, axes = plt.subplots(2, 2, figsize=(18, 14))
    fig.suptitle('Temporal Patterns in NYC Taxi Trips', fontsize=18, fontweight='bold')
    
    # Hourly patterns
    if not temporal_patterns['hourly'].empty:
        axes[0, 0].bar(temporal_patterns['hourly'].index, temporal_patterns['hourly'].values, 
                       color='skyblue', alpha=0.8, width=0.8)
        axes[0, 0].set_title('Trips by Hour of Day', fontsize=14)
        axes[0, 0].set_xlabel('Hour (0-23)', fontsize=12)
        axes[0, 0].set_ylabel('Number of Trips', fontsize=12)
        axes[0, 0].set_xticks(np.arange(0, 24, 2))
        axes[0, 0].grid(axis='y', linestyle='--', alpha=0.7)
    else:
        axes[0,0].text(0.5, 0.5, 'No hourly data', horizontalalignment='center', verticalalignment='center', transform=axes[0,0].transAxes)

    # Daily patterns
    day_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    if not temporal_patterns['daily'].empty:
        axes[0, 1].bar(temporal_patterns['daily'].index, temporal_patterns['daily'].values, 
                       color='lightcoral', alpha=0.8, width=0.8)
        axes[0, 1].set_title('Trips by Day of Week', fontsize=14)
        axes[0, 1].set_xlabel('Day of Week', fontsize=12)
        axes[0, 1].set_ylabel('Number of Trips', fontsize=12)
        axes[0, 1].set_xticks(range(len(day_names)))
        axes[0, 1].set_xticklabels(day_names)
        axes[0, 1].grid(axis='y', linestyle='--', alpha=0.7)
    else:
        axes[0,1].text(0.5, 0.5, 'No daily data', horizontalalignment='center', verticalalignment='center', transform=axes[0,1].transAxes)

    # Monthly patterns
    if not temporal_patterns['monthly'].empty:
        month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
        # Ensure index is int for mapping
        monthly_idx = temporal_patterns['monthly'].index.astype(int)
        axes[1, 0].bar(monthly_idx, temporal_patterns['monthly'].values, 
                       color='lightgreen', alpha=0.8, width=0.8)
        axes[1, 0].set_title('Trips by Month', fontsize=14)
        axes[1, 0].set_xlabel('Month', fontsize=12)
        axes[1, 0].set_ylabel('Number of Trips', fontsize=12)
        axes[1, 0].set_xticks(np.arange(1, 13))
        axes[1,0].set_xticklabels([month_names[i-1] for i in np.arange(1,13)], rotation=45)
        axes[1, 0].grid(axis='y', linestyle='--', alpha=0.7)
    else:
        axes[1,0].text(0.5, 0.5, 'No monthly data', horizontalalignment='center', verticalalignment='center', transform=axes[1,0].transAxes)
        
    # Heatmap of hour vs day of week
    if not temporal_patterns['hour_dow'].empty:
        sns.heatmap(temporal_patterns['hour_dow'], ax=axes[1, 1], cmap='YlOrRd', 
                    xticklabels=day_names, cbar_kws={'label': 'Number of Trips'})
        axes[1, 1].set_title('Trips Heatmap: Hour vs Day of Week', fontsize=14)
        axes[1, 1].set_xlabel('Day of Week', fontsize=12)
        axes[1, 1].set_ylabel('Hour of Day', fontsize=12)
    else:
        axes[1,1].text(0.5, 0.5, 'No Hour vs DoW data', horizontalalignment='center', verticalalignment='center', transform=axes[1,1].transAxes)
        
    plt.tight_layout(rect=[0, 0, 1, 0.96]) # Adjust layout to make space for suptitle
    plt.show()

    # Print summary statistics
    print("\n⏰ Temporal Pattern Summary:")
    if not temporal_patterns['hourly'].empty:
        print(f"Peak hour: {temporal_patterns['hourly'].idxmax()}:00 ({temporal_patterns['hourly'].max():,} trips)")
    if not temporal_patterns['daily'].empty:
        print(f"Busiest day: {day_names[temporal_patterns['daily'].idxmax()]} ({temporal_patterns['daily'].max():,} trips)")
    if not temporal_patterns['monthly'].empty:
        peak_month_idx = temporal_patterns['monthly'].idxmax()
        print(f"Peak month: {month_names[peak_month_idx-1]} ({temporal_patterns['monthly'].max():,} trips)")
else:
    print("No temporal data to display or summarize.")

ValueError: The columns in the computed data do not match the columns in the provided metadata.
  Extra:   []
  Missing: ['RateCodeID']

## Geospatial Analysis: H3 Aggregations and Visualization

In [ ]:
# Geospatial Aggregation by H3 Zones
def aggregate_by_h3(ddf, h3_resolution):
    """Aggregate trip counts by H3 zone."""
    h3_col = f'pickup_h3_r{h3_resolution}'
    if ddf is None or h3_col not in ddf.columns:
        logger.warning(f"H3 column {h3_col} not found or DDF is None. Skipping H3 aggregation.")
        return pd.DataFrame(columns=[h3_col, 'trip_count'])
        
    logger.info(f"📊 Aggregating trip counts by H3 resolution {h3_resolution}...")
    
    # Filter out null H3 values before grouping
    valid_h3_ddf = ddf.dropna(subset=[h3_col])
    if len(valid_h3_ddf) == 0:
        logger.warning(f"No valid H3 data for resolution {h3_resolution} after dropping NaNs.")
        return pd.DataFrame(columns=[h3_col, 'trip_count'])
        
    h3_counts = valid_h3_ddf.groupby(h3_col).size().compute()
    h3_counts_df = h3_counts.reset_index(name='trip_count').sort_values('trip_count', ascending=False)
    
    logger.info(f"✅ Aggregation complete. Found {len(h3_counts_df)} unique H3 zones.")
    return h3_counts_df

h3_main_res_col = f'pickup_h3_r{config.H3_MAIN_RESOLUTION}'
if processed_ddf is not None and h3_main_res_col in processed_ddf.columns:
    h3_aggregated_pickups = aggregate_by_h3(processed_ddf, config.H3_MAIN_RESOLUTION)
    print(f"\nTop H3 Zones (Resolution {config.H3_MAIN_RESOLUTION}) by Pickup Count:")
    if not h3_aggregated_pickups.empty:
        display(h3_aggregated_pickups.head())
    else:
        print("No H3 aggregated data to display.")
else:
    print(f"Skipping H3 aggregation as processed_ddf is None or H3 column '{h3_main_res_col}' is missing.")
    h3_aggregated_pickups = pd.DataFrame()

In [ ]:
# H3 Hexagon Geometry for Visualization
def h3_to_geodataframe(h3_df, h3_col):
    """Convert a DataFrame with H3 IDs to a GeoDataFrame with hexagon geometries."""
    if h3_df.empty or h3_col not in h3_df.columns:
        logger.warning("Input DataFrame for H3 to GeoDataFrame is empty or H3 column missing.")
        return gpd.GeoDataFrame()

    def get_h3_boundary(h3_id):
        try:
            # h3.h3_to_geo_boundary returns list of (lat, lon) tuples
            # Polygon expects (lon, lat)
            return Polygon([(lon, lat) for lat, lon in h3.h3_to_geo_boundary(h3_id, geo_json=False)])
        except Exception as e:
            logger.debug(f"Error getting boundary for H3 ID {h3_id}: {e}")
            return None

    h3_df['geometry'] = h3_df[h3_col].apply(get_h3_boundary)
    h3_gdf = gpd.GeoDataFrame(h3_df, geometry='geometry', crs='EPSG:4326')
    h3_gdf = h3_gdf.dropna(subset=['geometry']) # Remove rows where boundary creation failed
    return h3_gdf

# Folium Map - H3 Choropleth
def plot_h3_choropleth(h3_counts_df, h3_col='pickup_h3_r8', value_col='trip_count', map_title='NYC Taxi Pickups by H3 Zone'):
    """Create a Folium choropleth map for H3 aggregated data."""
    if h3_counts_df.empty:
        logger.warning("H3 counts data is empty. Cannot generate choropleth map.")
        print("Cannot generate H3 choropleth: data is empty.")
        return None

    logger.info(f"🗺️ Generating H3 Choropleth map for {value_col}...")
    
    # Convert to GeoDataFrame
    h3_gdf = h3_to_geodataframe(h3_counts_df.copy(), h3_col)
    if h3_gdf.empty:
        logger.warning("GeoDataFrame from H3 data is empty. Cannot generate choropleth map.")
        print("Cannot generate H3 choropleth: GeoDataFrame is empty after H3 conversion.")
        return None

    # Create Folium map centered on NYC
    nyc_center = [np.mean([config.NYC_BOUNDS['lat_min'], config.NYC_BOUNDS['lat_max']]), 
                  np.mean([config.NYC_BOUNDS['lon_min'], config.NYC_BOUNDS['lon_max']])]
    m = folium.Map(location=nyc_center, zoom_start=10, tiles='CartoDB positron')

    # Add choropleth layer
    try:
        folium.Choropleth(
            geo_data=h3_gdf.to_json(), # Convert GeoDataFrame to GeoJSON string
            data=h3_gdf,
            columns=[h3_col, value_col],
            key_on=f'feature.properties.{h3_col}', # Path to H3 ID in GeoJSON properties
            fill_color='YlOrRd',
            fill_opacity=0.7,
            line_opacity=0.2,
            legend_name=f'{value_col.replace("_", " ").title()}',
            name=map_title
        ).add_to(m)
    except Exception as e:
        logger.error(f"Error creating Choropleth layer: {e}")
        print(f"Error creating Choropleth layer: {e}")
        # Attempt to add individual polygons if Choropleth fails (e.g., due to data scale issues)
        # This is a fallback and might be slow for many polygons.
        # Normalize data for color mapping
        # from matplotlib.colors import Normalize, to_hex
        # from matplotlib.cm import YlOrRd
        # norm = Normalize(vmin=h3_gdf[value_col].min(), vmax=h3_gdf[value_col].quantile(0.95)) # Cap at 95th percentile for better color scale
        # for _, row in h3_gdf.iterrows():
        #    folium.GeoJson(
        #        row.geometry.__geo_interface__,
        #        style_function=lambda x, val=row[value_col]: {
        #            'fillColor': to_hex(YlOrRd(norm(val))),
        #            'color': 'black',
        #            'weight': 0.5,
        #            'fillOpacity': 0.7
        #        },
        #        tooltip=f"{row[h3_col]}: {row[value_col]}"
        #    ).add_to(m)

    folium.LayerControl().add_to(m)
    logger.info("✅ H3 Choropleth map generated.")
    return m

if not h3_aggregated_pickups.empty:
    # Take a sample for map visualization if too many H3 cells to prevent browser freezing
    sample_h3_map_data = h3_aggregated_pickups.head(2000) # Limit to top N H3s for map
    h3_choropleth_map = plot_h3_choropleth(
        sample_h3_map_data, 
        h3_col=f'pickup_h3_r{config.H3_MAIN_RESOLUTION}', 
        value_col='trip_count',
        map_title=f'NYC Taxi Pickups by H3 Zone (Res {config.H3_MAIN_RESOLUTION}) - Top {len(sample_h3_map_data)} zones'
    )
    if h3_choropleth_map:
        display(h3_choropleth_map)
else:
    print("No H3 aggregated data to plot choropleth map.")

## Geospatial Analysis: Pickup Hotspots and Clusters

In [ ]:
# Folium Map - Pickup Heatmap and Marker Cluster
def plot_pickup_heatmap_markers(ddf, sample_frac_heatmap, sample_frac_markers, lat_col, lon_col):
    """Create a Folium map with a heatmap and marker clusters for pickup locations."""
    if ddf is None or len(ddf) == 0:
        logger.warning("DDF is None or empty. Skipping heatmap/marker plot.")
        print("Cannot generate heatmap/markers: data is empty or None.")
        return None
    if lat_col not in ddf.columns or lon_col not in ddf.columns:
        logger.warning(f"Lat/Lon columns ('{lat_col}', '{lon_col}') not found. Skipping heatmap/marker plot.")
        print(f"Cannot generate heatmap/markers: Lat/Lon columns ('{lat_col}', '{lon_col}') not found.")
        return None

    logger.info(f"🗺️ Generating Pickup Heatmap and Marker Cluster map...")
    
    # Sample data for visualization (Pandas DataFrame)
    logger.info(f"Sampling data for heatmap ({sample_frac_heatmap*100:.2f}%) and markers ({sample_frac_markers*100:.2f}%)...")
    
    # Ensure sampling is done on valid coordinates
    ddf_coords = ddf[[lat_col, lon_col]].dropna().persist() # Persist small selection
    
    if len(ddf_coords) == 0:
        logger.warning("No valid coordinate data after dropping NaNs. Skipping map.")
        print("No valid coordinate data for heatmap/markers.")
        return None

    sample_heatmap_df = ddf_coords.sample(frac=sample_frac_heatmap, random_state=42).compute()
    sample_markers_df = ddf_coords.sample(frac=sample_frac_markers, random_state=123).compute()
    
    logger.info(f"Heatmap sample size: {len(sample_heatmap_df)}, Marker sample size: {len(sample_markers_df)}")

    if sample_heatmap_df.empty and sample_markers_df.empty:
        logger.warning("Sampled data is empty. Cannot generate map.")
        print("Sampled data for heatmap/markers is empty.")
        return None

    # Create Folium map centered on NYC
    nyc_center = [np.mean([config.NYC_BOUNDS['lat_min'], config.NYC_BOUNDS['lat_max']]), 
                  np.mean([config.NYC_BOUNDS['lon_min'], config.NYC_BOUNDS['lon_max']])]
    m = folium.Map(location=nyc_center, zoom_start=11, tiles='CartoDB positron')

    # Add Heatmap layer
    if not sample_heatmap_df.empty:
        heat_data = [[row[lat_col], row[lon_col]] for index, row in sample_heatmap_df.iterrows()]
        HeatMap(heat_data, radius=10, blur=15, name="Pickup Heatmap").add_to(m)
    
    # Add Marker Cluster layer (FastMarkerCluster for better performance)
    if not sample_markers_df.empty:
        marker_locations = [[row[lat_col], row[lon_col]] for index, row in sample_markers_df.iterrows()]
        # Using FastMarkerCluster for potentially large number of points
        FastMarkerCluster(data=marker_locations, name="Pickup Clusters (Sampled)").add_to(m)
        # Alternative: standard MarkerCluster
        # marker_cluster = MarkerCluster(name="Pickup Clusters (Sampled)").add_to(m)
        # for lat, lon in marker_locations:
        #     folium.Marker([lat, lon]).add_to(marker_cluster)

    # Add Borough boundaries if available
    if 'boroughs' in geo_data and not geo_data['boroughs'].empty:
        folium.GeoJson(
            geo_data['boroughs'],
            name='NYC Boroughs',
            style_function=lambda x: {'fillColor': 'gray', 'color': 'black', 'weight': 1, 'fillOpacity': 0.1}
        ).add_to(m)
        
    folium.LayerControl().add_to(m)
    logger.info("✅ Pickup Heatmap and Marker Cluster map generated.")
    return m

if processed_ddf is not None:
    pickup_heatmap_map = plot_pickup_heatmap_markers(
        processed_ddf, 
        sample_frac_heatmap=config.SAMPLE_RATES.get('heatmap', 0.01), 
        sample_frac_markers=config.SAMPLE_RATES.get('visualization', 0.001), # Smaller sample for markers
        lat_col=config.PICKUP_LAT_COL, 
        lon_col=config.PICKUP_LON_COL
    )
    if pickup_heatmap_map:
        display(pickup_heatmap_map)
else:
    print("Skipping pickup heatmap/marker map as processed_ddf is None.")

In [ ]:
# Advanced Geospatial Analysis - DBSCAN Clustering for Hotspots
def find_hotspots_dbscan(ddf, sample_frac, lat_col, lon_col, eps=0.001, min_samples=50):
    """Find hotspots using DBSCAN clustering on a sample of pickup locations."""
    if ddf is None or len(ddf) == 0:
        logger.warning("DDF is None or empty. Skipping DBSCAN clustering.")
        return None, pd.DataFrame()
    if lat_col not in ddf.columns or lon_col not in ddf.columns:
        logger.warning(f"Lat/Lon columns ('{lat_col}', '{lon_col}') not found. Skipping DBSCAN clustering.")
        return None, pd.DataFrame()

    logger.info(f"🔥 Finding hotspots using DBSCAN (eps={eps}, min_samples={min_samples})...")
    logger.info(f"Sampling {sample_frac*100:.2f}% of data for DBSCAN...")

    # Sample data and select coordinates, drop NaNs
    coords_ddf = ddf[[lat_col, lon_col]].dropna().persist()
    if len(coords_ddf) == 0:
        logger.warning("No valid coordinate data after dropping NaNs for DBSCAN.")
        return None, pd.DataFrame()
        
    sample_df = coords_ddf.sample(frac=sample_frac, random_state=42).compute()
    if sample_df.empty:
        logger.warning("Sampled data for DBSCAN is empty.")
        return None, pd.DataFrame()
    
    logger.info(f"DBSCAN on {len(sample_df)} points.")
    coords = sample_df[[lat_col, lon_col]].values

    # Scale data (important for distance-based algorithms like DBSCAN)
    # Note: Scaling lat/lon might distort geographic distances if not handled carefully.
    # For DBSCAN, 'eps' is in the units of the input data. If using raw lat/lon,
    # eps=0.001 is approx 111 meters in latitude, variable in longitude.
    # scaler = StandardScaler()
    # coords_scaled = scaler.fit_transform(coords)
    # db = DBSCAN(eps=eps_scaled, min_samples=min_samples).fit(coords_scaled) # eps needs adjustment if scaled

    # Using Haversine distance for DBSCAN for geographic data
    # Convert degrees to radians for haversine metric
    kms_per_radian = 6371.0088 
    # eps in kilometers, convert to radians for haversine
    # Example: if eps is 0.1 km (100 meters)
    epsilon_geo = (eps / kms_per_radian) 

    db = DBSCAN(eps=epsilon_geo, min_samples=min_samples, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
    
    sample_df['cluster'] = db.labels_
    num_clusters = len(set(db.labels_)) - (1 if -1 in db.labels_ else 0)
    logger.info(f"✅ DBSCAN complete. Found {num_clusters} clusters and { (db.labels_ == -1).sum()} noise points.")

    # Plot clusters on a map
    if num_clusters > 0:
        nyc_center = [np.mean([config.NYC_BOUNDS['lat_min'], config.NYC_BOUNDS['lat_max']]), 
                      np.mean([config.NYC_BOUNDS['lon_min'], config.NYC_BOUNDS['lon_max']])]
        map_clusters = folium.Map(location=nyc_center, zoom_start=11, tiles='CartoDB positron')
        
        # Create a color palette for clusters
        # Exclude noise points (cluster == -1)
        unique_labels = sorted(sample_df['cluster'].unique())
        unique_labels = [l for l in unique_labels if l != -1]
        colors = sns.color_palette("husl", len(unique_labels)).as_hex()
        cluster_colors = {label: colors[i] for i, label in enumerate(unique_labels)}

        for index, row in sample_df.iterrows():
            if row['cluster'] != -1: # Only plot actual clusters, not noise
                folium.CircleMarker(
                    location=[row[lat_col], row[lon_col]],
                    radius=3,
                    color=cluster_colors[row['cluster']],
                    fill=True,
                    fill_color=cluster_colors[row['cluster']],
                    fill_opacity=0.7,
                    tooltip=f"Cluster {row['cluster']}"
                ).add_to(map_clusters)
        return map_clusters, sample_df[sample_df['cluster'] != -1]
    else:
        logger.info("No clusters found by DBSCAN.")
        return None, sample_df

if processed_ddf is not None:
    # Adjust eps (in km) and min_samples based on data density and desired granularity
    # eps=0.1 means points within 100m (approx) can belong to the same cluster.
    dbscan_map, clustered_data = find_hotspots_dbscan(
        processed_ddf, 
        sample_frac=config.SAMPLE_RATES.get('clustering', 0.005), 
        lat_col=config.PICKUP_LAT_COL, 
        lon_col=config.PICKUP_LON_COL,
        eps=0.1, # Epsilon in kilometers (e.g., 0.1 km = 100 meters)
        min_samples=20 
    )
    if dbscan_map:
        print("🗺️ DBSCAN Hotspot Clusters Map:")
        display(dbscan_map)
        print("\nSample of Clustered Data (excluding noise):")
        display(clustered_data.head())
        print(f"\nCluster sizes:\n{clustered_data['cluster'].value_counts().sort_index()}")
    else:
        print("DBSCAN did not produce a map (e.g., no clusters found or data was insufficient).")
else:
    print("Skipping DBSCAN clustering as processed_ddf is None.")

## Trip Characteristics Analysis

In [ ]:
# Trip Characteristics Analysis (e.g., distance, fare, passenger count)
def analyze_trip_characteristics(ddf):
    """Analyze distributions of key trip characteristics."""
    if ddf is None or len(ddf) == 0:
        logger.warning("DDF is None or empty. Skipping trip characteristics analysis.")
        return {}

    logger.info("📊 Analyzing trip characteristics...")
    
    characteristics = {}
    cols_to_analyze = {
        'trip_distance': {'range': (0, 50)}, # miles, filter outliers for viz
        'fare_amount': {'range': (0, 100)}, # USD, filter outliers for viz
        'total_amount': {'range': (0, 150)}, # USD, filter outliers for viz
        'passenger_count': {'range': (0, 8)} # filter outliers for viz
    }
    
    # Compute basic stats for relevant columns
    desc_cols = [col for col in cols_to_analyze if col in ddf.columns]
    if not desc_cols:
        logger.warning("No characteristic columns found for analysis.")
        return {}
        
    # Persist relevant subset for multiple computations
    ddf_subset = ddf[desc_cols].dropna().persist()
    if len(ddf_subset) == 0:
        logger.warning("No data available for trip characteristics after dropping NaNs.")
        return {}
        
    characteristics['summary_stats'] = ddf_subset.describe().compute()

    # For histograms, sample data to avoid OOM with large datasets if not using Dask-Plotly
    # Or compute histograms with Dask then plot with Matplotlib/Seaborn
    sample_for_hist_df = ddf_subset.sample(frac=0.1, random_state=1).compute() # 10% sample
    if sample_for_hist_df.empty:
        logger.warning("Sampled data for histograms is empty.")
        return characteristics # Return summary stats at least
        
    fig, axes = plt.subplots(2, 2, figsize=(18, 12))
    axes = axes.flatten()
    fig.suptitle('Distribution of Trip Characteristics (Sampled Data)', fontsize=18, fontweight='bold')

    for i, (col, props) in enumerate(cols_to_analyze.items()):
        if col in sample_for_hist_df.columns:
            data_to_plot = sample_for_hist_df[col]
            # Apply range filtering for visualization clarity
            if 'range' in props:
                data_to_plot = data_to_plot[(data_to_plot >= props['range'][0]) & (data_to_plot <= props['range'][1])]
            
            if not data_to_plot.empty:
                sns.histplot(data_to_plot, kde=False, ax=axes[i], bins=50, color=sns.color_palette("viridis", 4)[i])
                axes[i].set_title(f'Distribution of {col.replace("_", " ").title()}', fontsize=14)
                axes[i].set_xlabel(col.replace("_", " ").title(), fontsize=12)
                axes[i].set_ylabel('Frequency', fontsize=12)
                axes[i].grid(axis='y', linestyle='--', alpha=0.7)
            else:
                axes[i].text(0.5, 0.5, f'No data for {col}', horizontalalignment='center', verticalalignment='center', transform=axes[i].transAxes)
        else:
            axes[i].text(0.5, 0.5, f'{col} not found', horizontalalignment='center', verticalalignment='center', transform=axes[i].transAxes)
            
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()
    
    logger.info("✅ Trip characteristics analysis complete.")
    return characteristics

if processed_ddf is not None:
    trip_char_analysis = analyze_trip_characteristics(processed_ddf)
    if 'summary_stats' in trip_char_analysis and not trip_char_analysis['summary_stats'].empty:
        print("\n📊 Summary Statistics of Trip Characteristics:")
        display(trip_char_analysis['summary_stats'])
    else:
        print("No summary statistics for trip characteristics to display.")
else:
    print("Skipping trip characteristics analysis as processed_ddf is None.")

In [ ]:
# Correlation Analysis
def analyze_correlations(ddf):
    """Compute and visualize correlation matrix for numerical features."""
    if ddf is None or len(ddf) == 0:
        logger.warning("DDF is None or empty. Skipping correlation analysis.")
        return None

    logger.info("🔗 Analyzing correlations...")
    
    # Select numerical columns for correlation
    numerical_cols = ddf.select_dtypes(include=np.number).columns.tolist()
    # Remove ID columns or others not suitable for direct correlation conceptually
    cols_to_exclude = ['VendorID', 'RatecodeID', 'payment_type', 
                       'pickup_hour', 'pickup_day_of_week', 'pickup_month'] # Temporal features are categorical-like
    cols_for_corr = [col for col in numerical_cols if col not in cols_to_exclude and 
                     config.PICKUP_LAT_COL not in col and config.PICKUP_LON_COL not in col and 
                     config.DROPOFF_LAT_COL not in col and config.DROPOFF_LON_COL not in col]

    if len(cols_for_corr) < 2:
        logger.warning(f"Not enough numerical columns ({cols_for_corr}) for correlation analysis.")
        return None
        
    # Compute correlation matrix (on a sample or full data if small enough)
    # Dask's .corr() can be memory intensive. For large data, sampling is advised.
    # Here, we try on full data, assuming 'processed_ddf' might be persisted or manageable.
    # If it's too large, this will be slow or fail. Consider sampling:
    # sample_corr_df = ddf[cols_for_corr].sample(frac=0.1).compute()
    # corr_matrix = sample_corr_df.corr()
    try:
        logger.info(f"Computing correlation matrix for columns: {cols_for_corr}")
        corr_matrix = ddf[cols_for_corr].corr().compute()
    except Exception as e:
        logger.error(f"Error computing correlation matrix: {e}. Trying with a sample.")
        try:
            sample_corr_df = ddf[cols_for_corr].dropna().sample(frac=0.01, random_state=1).compute()
            if len(sample_corr_df) < 2:
                 logger.error("Sample for correlation is too small.")
                 return None
            corr_matrix = sample_corr_df.corr()
        except Exception as e_sample:
            logger.error(f"Error computing correlation matrix on sample: {e_sample}")
            return None

    plt.figure(figsize=(12, 10))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
    plt.title('Correlation Matrix of Numerical Features', fontsize=16)
    plt.show()
    
    logger.info("✅ Correlation analysis complete.")
    return corr_matrix

if processed_ddf is not None:
    correlation_matrix = analyze_correlations(processed_ddf)
    if correlation_matrix is not None:
        print("\n🔗 Correlation Matrix:")
        display(correlation_matrix)
    else:
        print("Correlation analysis failed or no suitable data.")
else:
    print("Skipping correlation analysis as processed_ddf is None.")

## Interactive Visualizations with Plotly

In [ ]:
# Interactive Plotly Visualizations
def plotly_trip_analysis(ddf, sample_frac=0.01):
    """Create interactive Plotly visualizations for trip analysis."""
    if ddf is None or len(ddf) == 0:
        logger.warning("DDF is None or empty. Skipping Plotly visualizations.")
        return

    logger.info(f"🎨 Generating interactive Plotly visualizations (sample_frac={sample_frac})...")
    
    # Columns needed for plots
    plot_cols = ['trip_distance', 'fare_amount', 'passenger_count', 'pickup_hour']
    # Check if essential columns exist
    if not all(col in ddf.columns for col in ['trip_distance', 'fare_amount']):
        logger.warning("Essential columns for Plotly viz (trip_distance, fare_amount) are missing.")
        return
        
    # Sample data for Plotly (Pandas DataFrame for easier use with Plotly Express)
    # Persist subset before sampling
    ddf_subset = ddf[[col for col in plot_cols if col in ddf.columns]].dropna().persist()
    if len(ddf_subset) == 0:
        logger.warning("No data for Plotly viz after dropping NaNs.")
        return
        
    sample_df = ddf_subset.sample(frac=sample_frac, random_state=42).compute()
    if sample_df.empty:
        logger.warning("Sampled data for Plotly is empty.")
        return

    logger.info(f"Plotting with {len(sample_df)} sampled rows.")

    # 1. Scatter plot: Trip Distance vs. Fare Amount
    if 'trip_distance' in sample_df.columns and 'fare_amount' in sample_df.columns:
        fig1_title = 'Trip Distance vs. Fare Amount'
        color_col = 'passenger_count' if 'passenger_count' in sample_df.columns else None
        
        # Filter for reasonable values to make plot readable
        plot_df_fig1 = sample_df[
            (sample_df['trip_distance'] > 0) & (sample_df['trip_distance'] < 50) &
            (sample_df['fare_amount'] > 0) & (sample_df['fare_amount'] < 200)
        ]
        if not plot_df_fig1.empty:
            fig1 = px.scatter(
                plot_df_fig1, 
                x='trip_distance', 
                y='fare_amount', 
                color=color_col,
                title=fig1_title,
                labels={'trip_distance': 'Trip Distance (miles)', 'fare_amount': 'Fare Amount (USD)'},
                opacity=0.5,
                hover_data=plot_df_fig1.columns
            )
            fig1.show()
        else:
            logger.warning(f"No data for '{fig1_title}' after filtering.")

    # 2. Box plot: Fare Amount by Hour of Day
    if 'fare_amount' in sample_df.columns and 'pickup_hour' in sample_df.columns:
        fig2_title = 'Fare Amount Distribution by Pickup Hour'
        plot_df_fig2 = sample_df[
            (sample_df['fare_amount'] > 0) & (sample_df['fare_amount'] < 100)
        ]
        if not plot_df_fig2.empty:
            fig2 = px.box(
                plot_df_fig2, 
                x='pickup_hour', 
                y='fare_amount', 
                title=fig2_title,
                labels={'pickup_hour': 'Hour of Day', 'fare_amount': 'Fare Amount (USD)'},
                color='pickup_hour'
            )
            fig2.update_xaxes(type='category') # Treat hour as categorical
            fig2.show()
        else:
            logger.warning(f"No data for '{fig2_title}' after filtering.")
            
    # 3. 3D Scatter plot of pickup locations (if lat/lon and a value like fare_amount exist)
    # This requires more data and specific columns like pickup_latitude, pickup_longitude
    # For this example, we'll skip it to keep it general, but here's a template:
    # if all(c in sample_df.columns for c in [config.PICKUP_LAT_COL, config.PICKUP_LON_COL, 'fare_amount']):
    #    fig3 = px.scatter_3d(sample_df, x=config.PICKUP_LON_COL, y=config.PICKUP_LAT_COL, z='fare_amount',
    #                         color='fare_amount', title='3D Pickup Locations and Fare Amount')
    #    fig3.show()
    
    logger.info("✅ Plotly visualizations generated.")

if processed_ddf is not None:
    plotly_trip_analysis(processed_ddf, sample_frac=config.SAMPLE_RATES.get('visualization', 0.01))
else:
    print("Skipping Plotly visualizations as processed_ddf is None.")

## Conclusion and Key Findings

This notebook performed an enhanced exploratory data analysis of the NYC Taxi dataset, leveraging Dask for scalable computation, GeoPandas and H3 for geospatial analysis, and various libraries for advanced visualization.

**Key steps and observations include:**

1.  **Data Loading and Preprocessing:** Large datasets were efficiently handled using Dask. Data types were enforced, and datetime conversions were performed. H3 hexagonal indexing was added at multiple resolutions to facilitate geospatial aggregation, alongside temporal feature engineering (hour, day of week, month).

2.  **Data Quality Assessment:** Missing values and coordinate validity were assessed. This step is crucial for understanding data limitations and ensuring robustness of subsequent analyses. (Actual percentages depend on the specific dataset used).

3.  **Temporal Analysis:**
    *   Pickup patterns by hour, day of week, and month were analyzed, revealing peak and off-peak times. 
    *   A heatmap of trips by hour and day of the week provided a detailed view of demand fluctuations.
    *   *Specific findings (e.g., peak hours typically in the late afternoon/evening, higher demand on weekdays) would be listed here based on the actual data.*

4.  **Geospatial Analysis:**
    *   **H3 Aggregations:** Trip counts were aggregated by H3 zones, identifying high-traffic hexagonal areas. These were visualized on a choropleth map, offering a granular view of pickup density across NYC.
    *   **Pickup Hotspots:** Folium heatmaps and marker clusters visualized general pickup concentrations. 
    *   **DBSCAN Clustering:** A more advanced technique, DBSCAN, was applied to a sample of pickup locations to identify statistically significant clusters (hotspots), which were then plotted on an interactive map. This helps in pinpointing specific areas of high taxi activity.

5.  **Trip Characteristics Analysis:**
    *   Distributions of `trip_distance`, `fare_amount`, `total_amount`, and `passenger_count` were examined. 
    *   *Observations might include typical trip distances being short, common fare ranges, and most trips having 1-2 passengers.*
    *   A correlation matrix highlighted relationships between numerical features (e.g., strong correlation between `trip_distance` and `fare_amount`).

6.  **Interactive Visualizations:** Plotly Express was used to create interactive charts, such as scatter plots of trip distance vs. fare, and box plots of fare by hour, allowing for dynamic exploration of the data.

**Overall Insights:**
*The combination of temporal and geospatial analysis provides a rich understanding of taxi usage patterns. Identifying hotspots and peak times can be valuable for resource allocation, urban planning, and business strategy for taxi services.*

**Future Work:**
*   Origin-Destination (OD) analysis between H3 zones.
*   Analysis of dropoff patterns.
*   Integration with external datasets (e.g., weather, public events) to explore influencing factors.
*   Predictive modeling for demand forecasting.

In [ ]:
# Cleanup Dask client and cluster
def cleanup_dask():
    """Close Dask client and cluster if they exist."""
    logger.info("🧹 Cleaning up Dask resources...")
    global client, cluster
    try:
        if 'client' in globals() and client:
            client.close()
            logger.info("Dask client closed.")
        if 'cluster' in globals() and cluster:
            cluster.close()
            logger.info("Dask cluster closed.")
    except NameError:
        logger.info("No Dask client or cluster found to close.")
    except Exception as e:
        logger.error(f"Error during Dask cleanup: {e}")

cleanup_dask()
print("✅ Notebook execution complete and Dask resources cleaned up (if initialized).")